In [2]:
from tools.ground_removal import *
from tools.voxelization import *
from tools.pedestrians import *
from sklearn.cluster import DBSCAN
import numpy as np
from mayavi import mlab
%load_ext autoreload
%autoreload 2

In [19]:
pcl,_ = get_frame_without_ground(0)
mlab.points3d(pcl[:,0], pcl[:,1], pcl[:,2], mode='point')
mlab.show()

In [20]:
dynamic_points = get_dynamic_points(0.1, 0, 5)

0
1
2
3


In [22]:
mlab.points3d(dynamic_points[:,0], dynamic_points[:,1], dynamic_points[:,2], mode='point')
mlab.show()

In [3]:
clustering = DBSCAN(eps = 0.6, min_samples=8).fit(dynamic_points)
labels = (clustering.labels_ + 1) / (clustering.labels_.max() + 1)

In [4]:
mlab.points3d(dynamic_points[:,0], dynamic_points[:,1], dynamic_points[:,2], labels, mode='cube',scale_mode = 'none', scale_factor=0.2)
mlab.show()

## TODO: dynamic points delat pomocne pole kde 0 znaci staticky bod, 1 znaci dynamicky a nedelat if x in voxels
firstly use clustering to separate noise from dynamic points - will still get some noise and dynamic objects will form separate cluster for each time frame, use clustering again to connect close clusters and eliminate clusters with no close neighbour cluster

In [5]:
filtered_dynamic_points = dynamic_points[labels > 0]
filtered_labels = labels[labels > 0]
print("dynamic points " + str(len(filtered_labels)))

dynamic points 1136


In [6]:
clustering2 = DBSCAN(eps = 3.0, min_samples=30).fit(filtered_dynamic_points)
labels2 = (clustering2.labels_ + 1) / (clustering2.labels_.max() + 1)
filtered_dynamic_points2 = filtered_dynamic_points[labels2 > 0]
labels2 = labels2[labels2>0]

In [7]:
mlab.points3d(filtered_dynamic_points2[:,0], filtered_dynamic_points2[:,1], filtered_dynamic_points2[:,2], labels2, mode='cube',scale_mode = 'none', scale_factor=0.3)
mlab.show()

In [8]:
mlab.points3d(dynamic_points[:,0], dynamic_points[:,1], dynamic_points[:,2], mode='cube',scale_mode = 'none', scale_factor=0.1)
mlab.points3d(filtered_dynamic_points2[:,0], filtered_dynamic_points2[:,1], filtered_dynamic_points2[:,2], labels2, mode='cube',scale_mode = 'none', scale_factor=0.2)
mlab.show()

# TODO
leverage the fact that the static points (which still remains after clustering) do not change position during time

Ground truth dynamic points

In [9]:
pts, intens = get_frame(0)
dynamic_mask = get_dynamic_points_mask(0)
pts = pts[dynamic_mask]
print(dynamic_mask.sum())
#print(pts[dynamic_mask].shape)

for frame in range(1,15):
    p, i = get_frame(frame)
    m = get_dynamic_points_mask(frame)
    pts = np.concatenate((pts, p[m]))
    #intens = np.concatenate((intens, i))
    #dynamic_mask = np.concatenate((dynamic_mask, m))


330


In [14]:
mlab.points3d(dynamic_points[:,0], dynamic_points[:,1], dynamic_points[:,2], mode='cube',scale_mode = 'none', scale_factor=0.1)
mlab.points3d(pts[:,0], pts[:,1], pts[:,2], mode='cube', scale_mode = 'none', scale_factor=0.1, color=(1,0,0))
mlab.show()

In [9]:
dynamic_mask.shape

(122829,)

In [10]:
pts.shape

(122829, 3)

In [105]:
file_name = "0" * int(6 - (len(str(1)))) + str(1)
labels = np.fromfile(PATH + "labels/" + file_name + ".label", dtype=np.uint32)

# Detecting people

TODO: nejdriv dbscan s malou epsilon vzdalenosti a potom udelat voxelizaci a v kazdym voxelu koukat na to, jestli kdyz tam nazacatku byly body z jednoho shluku tak jeslti tam za nekolik snimku nejsou zadne body tohoto shluku

In [3]:
pts, intens = get_synchronized_frames_without_ground(0,10)
mlab.points3d(pts[:,0], pts[:,1], pts[:,2], pts[:,3], mode='point')
mlab.show()

FileNotFoundError: [Errno 2] No such file or directory: '../../semantic_kitti_data/sequences/05/velodyne/000000.bin'

In [178]:
# clustering = DBSCAN(eps = 0.4, min_samples=10).fit(pts[:,:4]) # clustering with time as one coordinate, TODO might be usefull?
clustering = DBSCAN(eps = 0.3, min_samples=100, n_jobs=-1).fit(pts[:,:3])
labels = (clustering.labels_ + 1) / (clustering.labels_.max() + 1)
print(f"formed {clustering.labels_.max() + 1} number of clusters")

formed 139 number of clusters


In [180]:
clustered_mask = labels > 0
mlab.points3d(pts[:,0][clustered_mask], pts[:,1][clustered_mask], pts[:,2][clustered_mask], labels[clustered_mask], mode='point')
mlab.show()

In each cluster, compute centroid (mean of points) in time t0, t1, t2 ... and compare how do they differ, if there is little to no movement/change - static cluster, otherwise dynamic

In [181]:
cl = clustering.labels_
first_frame = 0
num_of_frames = 10
dynamic_clusters = []
differences = []
for cluster in range(0, cl.max() + 1):

    centroids = []
    mask = cl == cluster
    cluster_points = pts[mask]
    #print(f"cluster {cluster} with {cluster_points.shape[0]} num of points")
    for time in range(first_frame, first_frame + num_of_frames):
        mask_time = cluster_points[:,3] == time
        time_points = cluster_points[mask_time]
        if time_points.shape[0] == 0:
            continue
        centroid = np.sum(time_points, axis=0)[:3] / time_points.shape[0]
        #print(centroid)
        #print("------------")
        centroids.append(centroid)

    # compute pairwise difference e.g: centroid[1] - centroid[0], centroid[2] - centroid[1]...
    average_difference = 0
    for i in range(len(centroids) - 1):
        average_difference += centroids[i + 1] - centroids[i]

    average_difference = average_difference / (len(centroids) - 1)
    print(f"cluster {cluster} {average_difference}")
    differences.append(average_difference)

cluster 0 [-0.00321994 -0.00385459  0.00816561]
cluster 1 [-0.00412364 -0.00916053  0.00210797]
cluster 2 [-0.00125728 -0.00637047  0.00466981]
cluster 3 [-0.00833369  0.00567463  0.00827979]
cluster 4 [ 0.00115354 -0.0112289   0.0044886 ]
cluster 5 [-0.00222893 -0.00450391  0.00383949]
cluster 6 [-0.00361377  0.00726656  0.00267026]
cluster 7 [-0.0029488   0.00468001  0.00517958]
cluster 8 [-0.00179695  0.00201502  0.00043335]
cluster 9 [ 0.0064451   0.00287207 -0.00491468]
cluster 10 [0.01141739 0.00343974 0.00186328]
cluster 11 [0.00937691 0.00336051 0.00258787]
cluster 12 [-0.00223744  0.0016092   0.01499641]
cluster 13 [ 0.00514632 -0.01061343 -0.00300665]
cluster 14 [-0.0048148   0.00183914  0.00125581]
cluster 15 [-0.00750609  0.00262548 -0.00457061]
cluster 16 [-0.00561076  0.00216127  0.0014158 ]
cluster 17 [-0.00890002  0.01107443 -0.00371223]
cluster 18 [-0.00704801  0.01223143 -0.00396048]
cluster 19 [0.03156877 0.00247604 0.00014293]
cluster 20 [-0.0098891   0.02466093  0.

In [182]:
differences = np.array(differences)
#dynamic_mask = differences[:,0] >= 0.11
#dynamic_clusters = np.argwhere(differences[:,0] >= 0.08)
dynamic_clusters = np.argwhere(np.linalg.norm(differences[:,0:2], axis=1) >= 0.08)
dynamic_clusters

array([[82]])

In [183]:
clustered_mask = clustering.labels_ == 82

mlab.points3d(pts[:,0][clustered_mask], pts[:,1][clustered_mask], pts[:,2][clustered_mask], color=(1,0,0), mode='point')
mlab.points3d(pts[:,0][~clustered_mask], pts[:,1][~clustered_mask], pts[:,2][~clustered_mask],  mode='point')
mlab.show()

In [86]:
mlab.points3d(pts[:,0], pts[:,1], pts[:,2], pts[:,3], mode='point')
mlab.show()

In [6]:
pts, intens = get_synchronized_frames_without_ground(600,10)
mlab.points3d(pts[:,0], pts[:,1], pts[:,2], pts[:,3], mode='point')
mlab.show()

In [96]:
#clustering = DBSCAN(eps = 0.4, min_samples=10).fit(pts[:,:4]) # clustering with time as one coordinate, TODO might be usefull?
clustering = DBSCAN(eps = 0.3, min_samples=100).fit(pts[:,:3])
labels = (clustering.labels_ + 1) / (clustering.labels_.max() + 1)
print(f"formed {clustering.labels_.max() + 1} number of clusters")

formed 121 number of clusters


For frame 610 to 620, eps = 0.3, min_samples=100 the person is cluster 30

In [108]:
clustered_mask = labels > 0
mlab.points3d(pts[:,0][clustered_mask], pts[:,1][clustered_mask], pts[:,2][clustered_mask], labels[clustered_mask], mode='point')
mlab.show()

In [124]:
clustered_mask = clustering.labels_ == 96

mlab.points3d(pts[:,0][clustered_mask], pts[:,1][clustered_mask], pts[:,2][clustered_mask], color=(1,0,0), mode='point')
#mlab.points3d(pts[:,0][~clustered_mask], pts[:,1][~clustered_mask], pts[:,2][~clustered_mask],  mode='point')
mlab.show()

In [225]:
labels = (clustering_labels + 1) / (clustering_labels.max() + 1)
clustered_mask = labels > 0
mlab.points3d(pts[:,0][clustered_mask], pts[:,1][clustered_mask], pts[:,2][clustered_mask], labels[clustered_mask], mode='point')
mlab.show()

In [223]:

#pts, clustering_labels, dynamic_clusters, differences = find_pedastrians(607,10)
pts, clustering_labels, dynamic_clusters, differences = find_pedastrians(600,10)
dynamic_clusters

formed 121 clusters
skipping cluster 1 at time 600 because of it's size w 0.6459751289230837, l 2.2671561316064412, h 0.8526234901182246
skipping cluster 2 because of it's size w 14.562508044020802, l 11.80480118965238, h 2.4394148048262685
skipping cluster 3 at time 600 because of it's size w 0.0, l 0.0, h 0.0
skipping cluster 5 at time 600 because of it's size w 0.6726115370896366, l 3.233745801255992, h 1.7321221656366985
skipping cluster 6 because of it's size w 8.94013532612044, l 21.33324458562666, h 3.056385482011585
skipping cluster 7 because of it's size w 0.638023414783305, l 1.1217913382049574, h 0.4567630620429153
skipping cluster 8 at time 600 because of it's size w 0.9472818957141556, l 4.0319781607415734, h 1.7246285751135009
skipping cluster 9 at time 604 because of it's size w 0.10373721282357451, l 0.16138770583836504, h 0.5425771645254693
skipping cluster 12 because of it's size w 1.7050813627448633, l 1.0700849506061445, h 2.5578891670624397
skipping cluster 13 beca

array([[96]])

In [206]:
differences[30]

array([0.01414606, 0.060895  , 0.00352182])

In [226]:
clustered_mask = clustering_labels == 96

mlab.points3d(pts[:,0][clustered_mask], pts[:,1][clustered_mask], pts[:,2][clustered_mask], pts[:,3][clustered_mask], mode='point')
mlab.points3d(pts[:,0][~clustered_mask], pts[:,1][~clustered_mask], pts[:,2][~clustered_mask],  mode='point')
mlab.show()

In [207]:
mask = clustering_labels == 30
cluster_points = pts[mask]

for time in range(50, 50 + 10):
    mask_time = cluster_points[:, 3] == time
    time_points = cluster_points[mask_time]
    if time_points.shape[0] == 0:
        print(f"time {time} skipping because no points; ")
        continue

    width = np.abs(np.max(time_points[:, 0]) - np.min(time_points[:, 0]))
    length = np.abs(np.max(time_points[:, 1]) - np.min(time_points[:, 1]))
    height = np.abs(np.max(time_points[:, 2]) - np.min(time_points[:, 2]))
    print(f"{time} size w {width}, l {length}, h {height}")

50 size w 0.8661628882726014, l 1.7221143415656002, h 1.2298518573051873
51 size w 0.8218521742316689, l 1.7148780068693625, h 1.229210662256
52 size w 0.6953317899130234, l 1.7144772586876584, h 1.2227393920557381
53 size w 0.7530803633510281, l 1.7571495864106481, h 1.2233574856666758
54 size w 0.7490417799436813, l 1.6545844664250708, h 1.229055237671976
55 size w 0.8081813760715839, l 1.7137193445393208, h 1.2266069733287908
56 size w 0.9223642979489668, l 1.765901671616696, h 1.2696274542175146
57 size w 0.7187853921443832, l 1.555132730697835, h 1.081137112755457
58 size w 0.5755600783926305, l 1.0700294126091165, h 0.8163401302292068
59 size w 0.45681619483812597, l 0.5698436320308105, h 0.667576489345703


array([-0.04741701,  0.00287904, -0.03914796])

0.08689443161746327

Ideas for centroid-difference-approach:
 * threshold each difference by maximum person movement speed
 * person's direction and speed will not change drastically between frame
 * measure average distance of all points from centroid - should be small with people
 * consider only clusters which are observable at all times - raycasting of some sort

In [176]:
centroids = {}
mask = clustering_labels == 4
cluster_points = pts[mask]

width = np.abs(np.max(cluster_points[:, 0]) - np.min(cluster_points[:, 0]))
length = np.abs(np.max(cluster_points[:, 1]) - np.min(cluster_points[:, 1]))
height = np.abs(np.max(cluster_points[:, 2]) - np.min(cluster_points[:, 2]))

In [177]:
print(width, length, height)


0.8501504247822709 0.9890869692921456 1.5789348384748871


In [178]:
message = ""
for time in range(603, 603 + 10):
    mask_time = cluster_points[:, 3] == time
    time_points = cluster_points[mask_time]
    if time_points.shape[0] == 0:
        message += f"cluster {93}, time {time} skipping because no points; "
        continue
    centroid = np.sum(time_points, axis=0)[:3] / time_points.shape[0]

    centroids[time] = centroid
    # centroids.append(centroid)
if len(message) > 0:
    print(message+"\n")

cluster 93, time 606 skipping because no points; cluster 93, time 607 skipping because no points; cluster 93, time 608 skipping because no points; cluster 93, time 609 skipping because no points; cluster 93, time 610 skipping because no points; cluster 93, time 611 skipping because no points; cluster 93, time 612 skipping because no points; 



In [179]:
centroids_keys = list(centroids.keys())
average_difference = 0
num_of_differences = 0
valid = True
for i in range(len(centroids) - 1):
    # only considering differences between consecutive time frames
    if valid and centroids_keys[i + 1] == centroids_keys[i] + 1:
        difference = centroids[centroids_keys[i + 1]] - centroids[centroids_keys[i]]
        norm_of_difference_xy = np.linalg.norm(difference[:2])
        if norm_of_difference_xy < MAX_MOVEMENT_SPEED_OF_PERSON:
            average_difference += difference
            num_of_differences += 1
        else:
            valid = False
            print(f"cluster {cluster} invalid because norm of difference at "
                  f"time {centroids_keys[i + 1]} and {centroids_keys[i]} is {norm_of_difference_xy} ")
            break


In [180]:
average_difference = average_difference / num_of_differences
print(average_difference)
print(np.linalg.norm(average_difference[:2]))

[-0.03894122  0.01619187 -0.01094214]
0.04217339222503359


In [139]:
mask = clustering_labels == 3
cluster_points = pts[mask]

for t in range(600, 610):
    time_mask = cluster_points[:,3] == t
    if np.sum(time_mask) == 0:
        print(f"no points at time {t}")
        continue
    mlab.points3d(cluster_points[:,0][time_mask], cluster_points[:,1][time_mask], cluster_points[:,2][time_mask], color=(1,0,0), mode='point')
    mlab.points3d(cluster_points[:,0][~time_mask], cluster_points[:,1][~time_mask], cluster_points[:,2][~time_mask], mode='point')
    mlab.show()

no points at time 600
no points at time 601
no points at time 602


Points time animation

In [217]:
mask = clustering_labels == 30
cluster_points = pts[mask]
time_mask = cluster_points[:,3] == 50
centroid = np.sum(cluster_points[time_mask], axis=0)[:3]/cluster_points[time_mask].shape[0]
mlab.points3d(cluster_points[:,0][time_mask][0], cluster_points[:,1][time_mask][0], cluster_points[:,2][time_mask][0], mode='point')
ctrd = mlab.points3d(centroid[0], centroid[1], centroid[2],color=(1,0,0), mode='cube', scale_factor = 0.03)
plt = mlab.points3d(cluster_points[:,0][time_mask], cluster_points[:,1][time_mask], cluster_points[:,2][time_mask], mode='point')

@mlab.animate(delay=1000)
def anim():
    while True:
        for t in range(50, 50 + 10):
            time_mask = cluster_points[:,3] == t
            if np.sum(time_mask) == 0:
                print(f"no points at time {t}")
                continue
            print(t)
            centroid = np.sum(cluster_points[time_mask], axis=0)[:3]/cluster_points[time_mask].shape[0]
            ctrd.mlab_source.reset(x=centroid[0], y=centroid[1],z=centroid[2])
            plt.mlab_source.reset(x = cluster_points[:,0][time_mask], y=cluster_points[:,1][time_mask], z=cluster_points[:,2][time_mask])
            yield
anim()
mlab.show()

50
51
52
53
54
55
56
57
58
59
50


In [142]:
cluster_points

array([[ 149.1539157 , -160.43690353,    6.21348124,  603.        ],
       [ 149.18061161, -160.48379819,    6.2153496 ,  603.        ],
       [ 149.19487281, -160.52522811,    6.21652421,  603.        ],
       ...,
       [ 150.03739786, -160.91828667,    4.6724265 ,  612.        ],
       [ 150.0201334 , -160.96105774,    4.66845791,  612.        ],
       [ 150.02947617, -160.9908454 ,    4.66995182,  612.        ]])